# Veracity Project: Distributed Ledgers and Prolog
## Hayden McAlister - University of Otago 2022

### Running this demonstration
This demonstration is (currently) hosted on [Catalyst Cloud](https://catalystcloud.nz/). However, the IP address associated with this project is floating and could change regularly. If desired, this could be assigned some fixed domain name so this demonstration could be accessed anywhere, anytime.

Otherwise, this demonstration is accessible from GitHub as a docker-compose system. [This repository](TODO) describes how to download, run, and interact with this demonstration locally. 

---

## The infrastructure

In this research project we investigated how distributed ledger technology could be used in a veracity context to verify claims and reach consensus on the state of a system. Other research was done concurrently into using Prolog to describe the state of a system and look at provenance in an organ donation scenario. I built the infrastructure to integrate this Prolog into a jupyter notebook (for accessibility and easy interaction) and expose the blockchain for use.

We agreed that for a proof of concept a small scale, locally hosted blockchain would be most suitable. There is no need to put our tests on a "real" ledger at this stage, but if this project moves forward in this direction it may be useful to put information on a public ledger so it can be verified.

My first task was to find a suitable blockchain technology that could be locally hosted. I found [Hyperledger](https://www.hyperledger.org/) to be very promising, as the opensource projects could be locally hosted and seemed very flexible to cover many situations. I looked in to using Hyperledger [Indy](https://www.hyperledger.org/use/hyperledger-indy), [Aries](https://www.hyperledger.org/use/aries), [Besu](https://www.hyperledger.org/use/besu), and [Ursa](https://www.hyperledger.org/use/ursa) but determined that [Hyperledger Iroha](https://www.hyperledger.org/use/iroha) would be most suitable for my needs. This is mainly due to Iroha having the best documentation of the projects I looked at, and was the easiest to start developing with quickly. However, other projects could absolutely be used and in future local hosting may not even be needed.

I spent several weeks setting up infrastructure using Iroha and Docker. I managed to get four Iroha nodes running and communicating in separate containers, which is significant as four nodes is the minimum necessary to detect and correct for a single faulty node (crash fault or byzantine fault). While this demonstration does not look any further into faulty nodes, this infrastructure would support probing in this direction.

We can see these containers running. If running this demonstration locally, you can run 

`docker container ls`

and spot the containers labeled `iroha1` through `iroha4`. These containers are hosting the blockchain nodes and are constantly communicating to see if new transactions have arrived. These nodes are backed by the containers `some-postgres1` through `some-postgres4`. I am sure with enough convincing the postgres containers could be consolidated into a single container with several logins (one for each Iroha node), however this infrastructure was the easiest to develop quickly. 

We can also see what the current state of the blockchain is by running the following command. We will discuss this command later on, but for now just know it will query `iroha1` and get all blocks on the chain, storing them in `logs/blocks.log`.

In [4]:
%python
from IrohaUtils import *
log_all_blocks("blocks.log")

We can see in this file that the blocks are represented in JSON format, and are mostly incomprehensible jargon. A lot of this can be understood by trawling through [the documentation](https://iroha.readthedocs.io/en/main/), however this is time consuming and very dense (I said the documentation was the best of the projects above, I did not claim the documentation was clear). For this reason, I have developed some tools we will discuss soon that make developing and interacting with the Iroha network much easier. 

You may also notice other containers running due to this network. There is the `swipl-notebook` container which, unsurprisingly, runs the jupyter notebook server you are interacting with now! There is also a redis container which we will discuss later on.

Because the goal of this project was to eventually host some Prolog work, we had to get Prolog running in a jupyter notebook. Thankfully this task was done for us (and can be found [here](https://github.com/veracitylab/jupyter-swi-prolog)), and with some scripting the jupyter notebook can run Prolog code! The kernel essentially runs Prolog line by line through a python interface with SWIPL, then dumps the factbase into a file for consultation later. These files are referenced by Prolog when querying the factbase, so the files store the state of the world. This will be useful later.

---

### Kernel modifications

The kernel, while fantastic, needed extending to work with the blockchain network. I added some extra functionality which made development of Prolog with the blockchain a little easier.

First, the kernel checks environment variables to determine state. This means we can easily turn on or off functionality as needed. The supported variables are:

- BLOCKCHAIN: 0 or 1. Boolean check to interact with the blockchain or not. If 1 then after executing a Prolog cell the kernel will hash the fact file and store this hash on the blockchain. This provides proof that a cell was run and was added to a fact base.
- REDIS: 0 or 1. Boolean check to interact with the redis container or not. If 1 then after execution of a Prolog cell the kernel will map the hash of the fact file with the fact file text itself. This means that if a hash is known (say, if we found it on the blockchain) then we can retrieve the file that generated it. Currently, these files are stored unencrypted
- TIMESTAMPING: 0 or 1. Boolean check to add a timestamp to the fact files before hashing. If 0, no timestamp is added. This means that if a cell is run multiple times then only the first fact file can be stored. In particular, this means that if a cell undergoes a change from state A to B to A, then the blockchain will be unable to store the change back to state A. 
- LOGGING_LEVEL: Integer from 0 to 50. Determines the level of logging. The most useful levels are 10 (DEBUG) and 20 (INFO)

It would be nice to set these environment variables from within a notebook, so I added functionality to do this using "magic command" syntax. Starting a cell with `%ENV` will allow you to set environment variables as needed.

In [7]:
%ENV
BLOCKCHAIN=0
REDIS=1
TIMESTAMPING=1
LOGGING_LEVEL=20

SET ENV BLOCKCHAIN=0
SET ENV REDIS=1
SET ENV TIMESTAMPING=1
SET ENV LOGGING_LEVEL=20

Other magic commands are supported. For example, to change the fact file when running Prolog, we can use the `%file` command: (note that fact files are stored under the `consulted_files` directory.

In [8]:
%file: foo.pl

man(socrates).
mortal(X) :- man(X).

Redis storing b3a26cf295a4cd33d6e17b94ce65d18f@swipluser-foo-hash

We can use `%python` to run python code in a Prolog notebook:

In [6]:
%python
x=1
y=2
print(f"{x+y=}")

x+y=3

And we can consult files from the redis database using `%consult`:

In [9]:
%consult
b3a26cf295a4cd33d6e17b94ce65d18f@swipluser-foo-hash

b3a26cf295a4cd33d6e17b94ce65d18f@swipluser-foo-hash
% 1643674741031871443
man(socrates).
mortal(X) :- man(X).
--------------------------------------------------------------------------------
Successfully consulted foo.pl
--------------------------------------------------------------------------------


Using these extra kernel functions we can easily develop some example scenarios to demonstrate how this technology could be used in a veracity context

---

### Iroha Utilities

Before we move on to demonstrations, we need to discuss the Iroha utilities built for this project. These utilities form the backbone of the examples and streamline the process of working with Iroha. As discussed above, the Iroha blockchain is moderately well documented, but creating applications with this documentation takes a long time. The IrohaUtils package (located under `IrohaUtils`) is designed to make interactions with the blockchain infrastructure much easier. Full documentation of this package can be found under the `IrohaUtils/pydocs` directory.

When working with file hashes, we will also use IrohaHashCustodian module. This module handles the creation of domains and sending of hashes to the blockchain, as well as checking if hashes are present on the chain. It is important to note that this module relies heavily on querying the blockchain to check for the existence of hashes, which can become very slow for larger chains if the entire chain is queried for each hash. To avoid this, IrohaHashCustodian uses a BlockStorehouse that caches all blocks locally as they are seen, so the query can be run on local blocks before asking the chain for new blocks. This significantly speeds up the query process at the cost of a little space, but is well worth it.

Given more time, I would like to expand these modules a bit and make them available using pip install or otherwise. I would also like to add better caching functionality to the BlockStorehouse as currently all blocks are stored forever, which is not ideal for very large chains. Also, the admin user is exposed in IrohaUtils, making this module not suitable for production use, so I would like to implement something more secure. However, for a simple demonstration the modules work fine.

---

### Putting it all together

Finally, we have seen enough of the infrastructure and modules that we can get a broad overview of how this project works. Assuming that `BLOCKCHAIN=1`, `TIMESTAMPING=1` and `REDIS=1`, the intended process for interaction is:
- At startup of the kernel the user blockchain data (private key) is loaded and an IrohaHashCustodian is created. 
- A Prolog cell is created and run, defining some facts about a system (the local kernel believes these facts but a different kernel does not)
- Upon running the cell, the SWIPL kernel timestamps the file (adding the current time to the start of the file) and hashes the file. A new domain on the blockchain is created by the IrohaHashCustodian, and the resulting file hash is stored on the blockchain.
- The kernel then stores the file hash in the redis database, mapping the hash to the file itself.

These final three points can be repeated any number of times, creating any number of files and adding these to the factbase. After the local user has finished creating files, we know all of these file hashes are stored on the block chain and the file hashes are mapped (via redis) to the original files. If the local user then runs some arbitrary query on this factbase, they will get some result. Exciting! But so far nothing exceptional has happened.

Now consider what happens if a different user (User B) wants to verify the original users (User A) query. This other user must know the entire factbase of User A before running the query, but asking User A directly could result in compromised communication or corruption somewhere along the path. User B could therefore:

- Start the kernel, loading User B login data and creating an IrohaHashCustodian
- Query the blockchain for all hashes in the domain User A created
- With this list of hashes, User B can run the %CONSULT magic cell command
- The kernel will now take each hash and feed it through the redis database, converting hashes back to files, and loading these files into the fact base
- User B can now run the query with the same factbase as User A, getting the same results

Of course there is still some issues with this method. Communication could still be compromised or corrupted between the blockchain or redis database and User B. The redis database could map from the hash to an incorrect file (particularly if User A is malicious in storing the file hash). The process overall is much more complicated. 

However, the upsides are very important! Because the blockchain is queried rather than an individual user we can be sure that *any* user will get the same query results, while if User A is queried they could change answers based on who is querying. If encryption of files is implemented in the redis database then we can be sure that the correct file is stored in the data base (i.e. the file is encrypted by User A so must be decrypted using their public key, so User B can be sure the file is both stored by User A and has the correct hash). Finally, we should consider involving more than two users. If we had several users, all considered authoritative on some selection of facts, we could suppose that each user may only commit facts that they are authoritative over. Each other user could then collect the committed hashes and apply only those files with facts from authoritative users. This would mean we have a system where some users can set some facts and other users other facts, but *all* users can see *all* facts and agree on them.

As an example, in an organic supply chain we could state that a farmer could assert facts about their farming practices, a transporter could assert facts about transportation, a supermarket could assert facts about storage, and a consumer could collect all of these facts to determine the organic status of a product. Because these facts are committed to the blockchain all users can be sure that their factbase comes from authoritative sources, all assertions are public and agreed upon, and all users have the same factbase.  

---

### Demonstrations

Now that we have discussed how blockchain technology fits into this project, we should look at some demonstrations.

- `prolog-demonstration.ipynb` is a notebook showing how Prolog cells can be run in a notebook, then showing that those Prolog fact files can be stored on the blockchain, and finally how the blockchain can be queried to get file hashes from a domain.
- `hash-consult.ipynb` shows how a Prolog cell can be logged on the blockchain then later called from the chain and inserted into the factbase without needing to run the cell locally.
- `blockchain-testing.ipynb` shows some simple testing of the blockchain to demonstrate that the chain is secure and reliable, even in the face of malicious users.
